In [ ]:
!pip install -q torch
!pip install -q transformers
!pip install -q accelerate
!pip install -q sentence-transformers
!pip install -q langchain
!pip install -q langchain-community
!pip install -q sentence-transformers
!pip install -q tranfromers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the req

In [ ]:
import os
import pandas as pd

from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

from time import process_time

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

def get_lyrics_embedding(lyrics:str):
  return model.encode([lyrics.lower()])

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
lyrics_embeddigs = []
genres = []
unique_genres = set()

import string
songs_and_lyrics = pd.read_csv('./dataset.csv')

for idx in range(len(songs_and_lyrics)):
  genre = songs_and_lyrics["genre"][idx]
  unique_genres.add(genre)

  lyrics_as_bag_of_words_string = str(songs_and_lyrics["lyrics"][idx])

  embedding =  get_lyrics_embedding(lyrics_as_bag_of_words_string)[0]
  lyrics_embeddigs.append(embedding)
  genres.append(genre)



In [ ]:
train_set = pd.DataFrame({"genres":genres})
embeddings_list = []

embedding_size = len(lyrics_embeddigs[0])

for i in range(embedding_size):
  dim_i = pd.Series([embedding[i] for embedding in lyrics_embeddigs])
  embeddings_list.append(dim_i)

embeddings_df = pd.concat(embeddings_list, axis=1)
train_set = pd.concat([embeddings_df, train_set], axis=1)

In [ ]:
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
stop_words = set(stopwords.words('english'))
symbols = set([",",".","/","#","*","(",")","?","+","=",":",";","%","[", "]"])
song_specific = set(["chorus", "repeat", "verse", "oh", "ooh", "ah", "aah"])

def check_if_not_to_be_removed(token:str):
  return (not token.lower() in stop_words) and (not token.lower() in symbols) and (not token.lower() in song_specific) and (not any(char.isdigit() for char in token)) and (not "'" in token)

def clean_lyrics(lyrics:str):
  tokenized_lyrics = word_tokenize(lyrics)
  filtered_lyrics = [w for w in tokenized_lyrics if check_if_not_to_be_removed(w)]

  lyrics_concat=""
  for token in filtered_lyrics:
    lyrics_concat+=f" {token} "

  return lyrics_concat

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

baseline_model = KNeighborsClassifier(10, weights='distance')
baseline_model.fit(train_set[[i for i in range(embedding_size)]], train_set['genres'])


KNeighborsClassifier(n_neighbors=10, weights='distance')

In [ ]:
def estimate_genre(model, song_lyrics):
  filtered_lyrics = clean_lyrics(song_lyrics)

  print(f"Filtered lyrics: {filtered_lyrics}")

  embedding = get_lyrics_embedding(filtered_lyrics)

  genre = model.predict(embedding)
  return genre


In [ ]:
lyrics="Ooh You can dance You can jive Having the time of your life Ooh, see that girl Watch that scene Digging the dancing queen Friday night and the lights are low Looking out for a place to go Where they play the right music Getting in the swing You come to look for a king Anybody could be that guy Night is young and the music's high With a bit of rock music Everything is fine You're in the mood for a dance And when you get the chance You are the dancing queen Young and sweet Only seventeen Dancing queen Feel the beat from the tambourine, oh yeah You can dance You can jive Having the time of your life Ooh, see that girl Watch that scene Digging the dancing queen You're a teaser, you turn 'em on Leave 'em burning and then you're gone Looking out for another Anyone will do You're in the mood for a dance And when you get the chance You are the dancing queen Young and sweet Only seventeen Dancing queen Feel the beat from the tambourine, oh yeah You can dance You can jive Having the time of your life Ooh, see that girl Watch that scene Digging the dancing queen Digging the dancing queen"
print(estimate_genre(baseline_model, lyrics))

Filtered lyrics:  Ooh  dance  jive  time  life  Ooh  see  girl  Watch  scene  Digging  dancing  queen  Friday  night  lights  low  Looking  place  go  play  right  music  Getting  swing  come  look  king  Anybody  could  guy  Night  young  music  high  bit  rock  music  Everything  fine  mood  dance  get  chance  dancing  queen  Young  sweet  seventeen  Dancing  queen  Feel  beat  tambourine  yeah  dance  jive  time  life  Ooh  see  girl  Watch  scene  Digging  dancing  queen  teaser  turn  Leave  burning  gone  Looking  another  Anyone  mood  dance  get  chance  dancing  queen  Young  sweet  seventeen  Dancing  queen  Feel  beat  tambourine  yeah  dance  jive  time  life  Ooh  see  girl  Watch  scene  Digging  dancing  queen  Digging  dancing  queen 
['pop']


In [ ]:
lyrics="Half past twelve And I'm watchin' the late show in my flat, all alone How I hate to spend the evening on my own Autumn winds Blowin' outside the window as I look around the room And it makes me so depressed to see the gloom There's not a soul out there No one to hear my prayer Gimme, gimme, gimme a man after midnight Won't somebody help me chase these shadows away? Gimme, gimme, gimme a man after midnight Take me through the darkness to the break of the day Movie stars Find the end of the rainbow with a fortune to win It's so different from the world I'm living in Tired of TV I open the window and I gaze into the night But there's nothing there to see, no one in sight There's not a soul out there No one to hear my prayer Gimme, gimme, gimme a man after midnight Won't somebody help me chase these shadows away? Gimme, gimme, gimme a man after midnight Take me through the darkness to the break of the day Gimme, gimme, gimme a man after midnight Gimme, gimme, gimme a man after midnight There's not a soul out there No one to hear my prayer Gimme, gimme, gimme a man after midnight Won't somebody help me chase these shadows away? Gimme, gimme, gimme a man after midnight Take me through the darkness to the break of the day Gimme, gimme, gimme a man after midnight Won't somebody help me chase these shadows away? Gimme, gimme, gimme a man after midnight Take me through the darkness to the break of the day"
print(estimate_genre(baseline_model, lyrics))

Filtered lyrics:  Half  past  twelve  watchin  late  show  flat  alone  hate  spend  evening  Autumn  winds  Blowin  outside  window  look  around  room  makes  depressed  see  gloom  soul  one  hear  prayer  Gim  gim  gim  man  midnight  Wo  somebody  help  chase  shadows  away  Gim  gim  gim  man  midnight  Take  darkness  break  day  Movie  stars  Find  end  rainbow  fortune  win  different  world  living  Tired  TV  open  window  gaze  night  nothing  see  one  sight  soul  one  hear  prayer  Gim  gim  gim  man  midnight  Wo  somebody  help  chase  shadows  away  Gim  gim  gim  man  midnight  Take  darkness  break  day  Gim  gim  gim  man  midnight  Gim  gim  gim  man  midnight  soul  one  hear  prayer  Gim  gim  gim  man  midnight  Wo  somebody  help  chase  shadows  away  Gim  gim  gim  man  midnight  Take  darkness  break  day  Gim  gim  gim  man  midnight  Wo  somebody  help  chase  shadows  away  Gim  gim  gim  man  midnight  Take  darkness  break  day 
['country']


In [ ]:
lyrics="You'll take my life but I'll take yours, too You'll fire your musket but I'll run you through So when you're waiting for the next attack You'd better stand, there's no turning back The bugle sounds, the charge begins But on this battlefield, no one wins The smell of acrid smoke and horses' breath As I plunge on into certain death Oh Oh The horse he sweats with fear, we break to run The mighty roar of the Russian guns And as we race towards the human wall The screams of pain as my comrades fall We hurdle bodies that lay on the ground And the Russians fire another round We get so near yet so far away We won't live to fight another day Oh Oh We get so close, near enough to fight When a Russian gets me in his sights He pulls the trigger and I feel the blow A burst of rounds take my horse below And as I lay there gazing at the sky My body's numb and my throat is dry And as I lay forgotten and alone Without a tear I draw my parting groan Oh Oh"
print(estimate_genre(baseline_model, lyrics))

Filtered lyrics:  take  life  take  fire  musket  run  waiting  next  attack  better  stand  turning  back  bugle  sounds  charge  begins  battlefield  one  wins  smell  acrid  smoke  horses  breath  plunge  certain  death  horse  sweats  fear  break  run  mighty  roar  Russian  guns  race  towards  human  wall  screams  pain  comrades  fall  hurdle  bodies  lay  ground  Russians  fire  another  round  get  near  yet  far  away  wo  live  fight  another  day  get  close  near  enough  fight  Russian  gets  sights  pulls  trigger  feel  blow  burst  rounds  take  horse  lay  gazing  sky  body  numb  throat  dry  lay  forgotten  alone  Without  tear  draw  parting  groan 
['blues']
